In [16]:
import pickle
import numpy as np
import time
import faiss
import json
import random
import sys

import torch.cuda
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
facts_path = "corpus.jsonl"
with open(facts_path) as f:
    fact_sets = [json.loads(line) for line in f.readlines()]
facts = [fact_set["text"] for fact_set in fact_sets]

# facts_path = "queries.jsonl"
# with open(facts_path) as f:
#     fact_sets = [json.loads(line) for line in f.readlines()]
# facts = [fact_set["text"] for fact_set in fact_sets]

KeyError: 'question'

In [3]:
len(facts)

97852

In [4]:
len(facts[0])

74

In [5]:
sys.getsizeof(facts)

800984

In [6]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name, device='cuda')

In [7]:
embeddings = model.encode(facts, device='cuda', show_progress_bar=True)

Batches:   0%|          | 0/3058 [00:00<?, ?it/s]

In [8]:
with open(f'emb_hotpotquestions_{model_name}.pickle', 'wb') as f:
    pickle.dump(embeddings, f, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
max_l, min_l = 0, 999999999999
max_words, min_words = 0, 999999999999
for fact in facts:
    max_l, min_l = max(max_l, len(fact)), min(min_l, len(fact))
    c = fact.count(" ")
    max_words, min_words = max(max_words, c), min(min_words, c)
print(max_l, min_l)
print(max_words, min_words)

8237 0
1377 0


In [9]:
index_flatip = faiss.IndexFlatIP(embeddings.shape[1])
index_flatip.add(embeddings)
sys.getsizeof(index_flatip)

48

In [11]:
quant = faiss.IndexFlatIP(embeddings.shape[1])
index_ivfflat = faiss.IndexIVFFlat(quant, embeddings.shape[1], 100)
index_ivfflat.train(embeddings)
index_ivfflat.add(embeddings)

In [14]:
with open(f'index_ivfflat_hotpotquestions_{model_name}.pickle', 'wb') as f:
    pickle.dump(index_ivfflat, f, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
with open(f'index_flatip_hotpot_{model_name}.pickle', 'rb') as f:
    index_flatip = pickle.load(f)
# with open(f'index_ivfpq_hotpot_{model_name}.pickle', 'rb') as f:
#     index_ivfpq = pickle.load(f)
with open(f'index_ivfflat_hotpot_{model_name}.pickle', 'rb') as f:
    index_ivfflat = pickle.load(f)

In [12]:
sys.getsizeof(index_ivfpq)

48

In [22]:
q1 = "Which magazine was started first Arthur's Magazine or First for Women?"
q2 = "Who killed the Tsar of Russia?"
q3 = "What language family is the language of the tribe of the man who instructed Jeff Ball in?"
question = q3
k = 3

D, I = index_flatip.search(model.encode([question]), k)
result_facts = [facts[i] for i in I[0]]
for fact in result_facts:
    print('- ' + fact)
print("-" * 100)

# D, I = index_ivfpq.search(model.encode([question]), k)
# result_facts = [facts[i] for i in I[0]]
# for fact in result_facts:
#     print('- ' + fact)
# print("-" * 100)

D, I = index_ivfflat.search(model.encode([question]), k)
result_facts = [facts[i] for i in I[0]]
for fact in result_facts:
    print('- ' + fact)

- The Dani–Kwerba languages were a hypothetical language family proposed by Arthur Capell in 1962 and adopted by Wurm as part of his Trans–New Guinea (TNG) phylum. Malcolm Ross reassigned the Dani languages to a West Trans–New Guinea linkage and the Kwerba languages to his Tor–Kwerba family, outside of TNG altogether.
- Derek Bickerton (born March 25, 1926) is a linguist and Professor Emeritus at the University of Hawaii, Manoa. Based on his work in creole languages in Guyana and Hawaii, he has proposed that the features of creole languages provide powerful insights into the development of language both by individuals and as a feature of the human species. He is the originator and main proponent of the language bioprogram hypothesis according to which the similarity of creoles is due to their being formed from a prior pidgin by children who all share a universal human innate grammar capacity.
- The Chickasaw language ("Chikashshanompa’", IPA [tʃikaʃːanompaʔ] ) is a Native American lang